# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break

    print('網頁更新中...')
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 銀行、券商互換負面信用資料　最快2020年第1季上線

▲金管會主委顧立雄。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
金管會主委顧立雄今（25）日表示，今年推動跨業別信用資料交換，台灣正比照日本CRIN信用資訊系統分享模式，研議讓銀行與證券商可查詢對方的負面信用資料，在聯徵中心多次協商後，預計明（2020）年第1季上線，相信對銀行業及證券業營運發展均有所助益。
金管會主委顧立雄今天出席第13屆金安獎及金質獎暨金優獎表揚大會致詞時表示，聯徵中心是台灣唯一完整蒐集與建置跨金融機構的信用資料服務機構，自成立40多年以來，提供會員信用紀錄及營運財務資訊，有效發展金融業徵信輔助機制，對於提升全國信用制度健全扮演關鍵角色。
為推動跨業別信用資料交換，顧立雄指出，今年邀集聯徵中心、銀行公會、證交所及證券商公會等機關，比照日本CRIN信用資訊系統分享模式，研議讓銀行業及證券業得以分享查詢不同業別負面信用資料，預計明年第1季上線。
金管會目前也推動「電子支付機構及電子票證發行機構管理法制整合」，顧立雄說，修正重點除擴大金流核心業務，並新增金流附隨業務及金流相關衍生業務，例如辦理國內外小額匯兌業務、紅利積點整合及折抵代理收付實質交易款項等，打造完整支付生態圈，同時開放跨機構金流服務，達到跨機構資金移轉及通路共享。
此外，為落實電子支付機構對收款使用者的風險控管政策，顧立雄表示，今年也已修訂電子支付機構業務管理規則，明定電子支付機構須把合作的商家資訊報送聯徵中心，以建立電子支付機構與收款使用者往來情形資料庫，提供電子支付機構查詢，並與信用卡收單機構交換利用該等資訊，更能促進交易的便利性。
----------------------------------------------------------------------

[1] 新台幣狂升逾1角！美元相對走弱　匯價創1年半以來新低

▲ 美元匯價跌破30.4元。（圖／路透社）
記者李瑞瑾／台北報導
新台幣表現強勢，也讓美元匯價創下1年半以來新低。根據台銀今（25）天下午牌告的美元現金賣出匯價，美元為30.355元，相較今年8月初的高點匯價，換新台幣5萬如今可以多拿到79美元，等

▲ 新台幣兌美元盤中強升1角。（圖／路透社）
記者李瑞瑾／台北報導
新台幣兌美元匯率維持高檔震盪，並在今（25）天盤中再度強升1角，最高來到30.105元兌1美元，創下去年6月15日（匯率30.002元兌1美元）、逾1年半以來新高。
台北外匯市場新台幣兌美元匯率今天開盤貶值0.5分、以30.21兌1美元開出，開盤後即轉為走揚。截至上午11點36分，新台幣匯率來到30.108元兌1美元、升值9.7分。
----------------------------------------------------------------------

[14] 製造業結束連兩月下滑轉為上揚　台經院：明年景氣偏樂觀

▲台經院院長張建一（左1）。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台經院今（25）日公布最新景氣動向調查，台經院指出受美中貿易戰干擾未歇、全球經濟疲弱影響，市場需求保守觀望，讓傳統產業廠商出口、生產與接單表現依舊不振，但近期美中貿易緊張情勢暫告緩解，有助於提振市場信心，根據調查結果顯示，製造業營業氣候測驗點結束先前連兩個月下滑態勢轉為上揚。台經院院長張建一更表示，如果2020年沒有意外，台灣整體經濟成長率是可以樂觀看待。
台經院表示，近期美中貿易緊張情勢暫告緩解，有助於提振市場信心，使得一些權重較大的化學工業、鋼鐵基本工業、石油及煤製品業與金屬製品業廠商對當月與未來半年景氣看法樂觀的比例高於10月調查，且先進製程持續發威、記憶體市況已顯好轉、高階半導體封測競爭力突出、中國去美國化趨勢持續等支撐下，使得電子零組件業看好未來半年景氣的廠商比例明顯較10月調查為高。
據調查顯示，11月製造業營業氣候測驗點結束先前連兩個月下滑態勢轉為上揚，服務業營業氣候測驗點雖連四個月下滑，然跌幅漸縮小，營建業測驗點則上揚轉為下滑。製造業廠商認為當11月景氣為「好」比率為18.4%，較10月21.1%減少2.7個百分點，認為當月景氣為「壞」的比率則為27.6%，較10月34.7%減少7.1個百分點。
因此整體而言，11月製造業營業氣候測驗點為93.42點，較上月修正後之92.13點增加1.29點。
張建一表示，今年的受惠於轉單效應影響，在經濟成長率的上比預計的好很多，雖然整體經濟不壞，但值得注意的為產業間的差異非常大，尤其是半導體、電子業等高階製程的部分皆表現不錯，雖然中美貿易戰趨

▲ 新台幣兌美元今天開盤貶值0.5分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（25）天開盤貶值0.5分、以30.21兌1美元開出。
新台幣兌美元昨（24）日收盤收在30.205兌1美元；今天開盤以30.21元開出、貶值0.5分。
開盤後新台幣轉為升值，截至上午9點07分，新台幣為30.15兌1美元，升值5.5分。
----------------------------------------------------------------------

[24] 外資放假去！台股狹幅震盪　開盤小漲2點開11,978點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
迎接西方聖誕假期，華爾街交投清淡，美股四大指數漲跌互見；台股少了外資挹注持續量縮震盪，今（25）天開盤小漲2點、以11,978點開出。
截至上午9點04分，指數來到11,989點、上漲18點。
電子三雄中，台積電（2330）平盤以332元開出；鴻海（2317）開在91.1元、上漲0.22%；股王大立光（3008）以4900元開出、上漲0.61%。
美國股市昨（24）日於下午提前休市，投資人交易量減少，美股四大指數小漲小跌，其中那斯達克指數則是連續9天創新高。
道瓊指數下跌36.08點，收28515.45點、跌幅0.13%；那斯達克指數上漲7.24點，收8952.88點、漲幅0.08%；標普 500 指數下跌0.63點，收3223.38點、跌幅0.02%；費城半導體指數上漲5.08點，收1862.2點、漲幅0.27%。
----------------------------------------------------------------------

[25] 與住戶同樂！太平別墅新落成高規格舉辦聖誕晚會

【企劃特輯】
當天參加的住戶都很開心，甚至表示：「有幾次買房的經驗，也有住大樓與透天別墅的經驗，大樓花管理基金辦中秋晚會是很普遍的，住透天社區就沒遇過舉辦活動。會買生活家建設的房子除了房子很美建材很吸引我外，就是因為從頭到尾都跟以往的經驗不同，以往看房看了半天也不知道怎麼蓋的，但生活家建設在施工上介紹的非常詳細，不怕比較！因為以往的買房經驗讓我會特別注意一些問題，但生活家非常貼心，不僅都事先替我們想到而且做得非常好，還告訴我內行人才懂的差異性，讓我有驚豔

目前台股在12000點左右盤整，許多投資人都期盼能盡早突破歷史高點12682。在《雲端最有錢》節目中，「獅公分析師」李永年提到當年12682點和這次12000點情況完全不一樣，上次一張股票甚至可以買超過一間房子，上攻類股主要是金融股、營建股、資產股，而這次主要是電子產業股。
李永年回顧當年股王國泰人壽，從1985年的每股40元起漲，最高價來到1975元，等於上漲49倍，如果在以當時每年配股5元計算，1985年買一張國泰人壽，1989年已經增加至5張，換算下來等於漲了246倍。對比目前股王大立光，從最低價漲到最高價，漲了40倍，歷時15年，反觀以前的國泰人壽僅花4年就漲了49倍，落差其實相當大。
的確，三位來賓都認為當年情況比現今還要好賺。不過當時股價一下跌，也讓不少投資人印堂發黑，李永年表示自己當時因為擔任證券公司投顧經理，市場敏感度高，所以早先一步逃跑。
李永年也提出一些當年股市崩盤前的徵兆，例如日本股市於1989年已反轉起跌，不過當時台股卻持續漲，這很明顯就怪異了；還有當時買股人氣沸騰，連出家人也瘋狂搶買，另外，當時主流股已露疲態，像是國泰人壽當時都不再創新高；還有，壓抑四年的四大基金（鴻運、國際...）展開飆升補漲行情。
但重點是台股目前是否有崩盤可能呢？李永年信心滿滿地表示，完全沒有崩盤徵兆，投資人可以放心。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[37] 金管會揪金控幕後藏鏡人　大股東持股申報2020年7月全都露

▲金管會為揪出金控、銀行背後的藏鏡人，預計明年7月1日起開始申報。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
金管會要揪出各金控、銀行背後的「藏鏡人」，要股權一切透明化，銀行局明（25）日發布申報應注意事項，要求申報實質受益人或最終控制人，並給予半年的緩衝期，將在明（2020）年7月起正式上路。
金管會銀行局完成修正「金融控股公司法第16條第2項持有已發行有表決權股份申報應注意事項」及「銀行法第25條第2項持有已發行有表決權股份申報應注意事項」，這次修正共有5大重點。
首先是參考「金融機構防制洗錢辦法」規定，增訂同一人、同一關係人或第三人為法人時，將實質受益人或最終控制權

▲國發會預計2020年導入人工智慧技術，以快速掌握景氣動態。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
國家發展委員會於今（24）日舉行「2019年終記者會」，針對明年的目標，主委陳美伶指出，內部已經擬定了進化、優化、基本功等三大目標，其中最特別的是，國發會為了要順應數位發展的趨勢、導入人工智慧，透過巨量資料與機器學習方法，更精確地提前掌握景氣動向的轉折，並適時提出因應對策。
陳美伶表示，現在的世界經貿情況是動態的，需要導入新科技來快速掌握，因此才會決定明年導入人工智慧技術，來加快認定景氣高峰與谷底的時程。
不過即便國發會導入人工智慧技術，未來也不會影響每月例行的景氣燈號發佈，陳美伶解釋，這是因為國發會每月的景氣燈號是公布上一個月的數據，而導入人工智慧是要提前掌握景氣動態，兩者並不衝突。
而除了導入人工智慧掌握景氣動態外，國發會明年也將以資料科學方法分析擘劃人口及人才政策；國發會指出，這是為更有效運用政府及民間各項人口及人力統計資料庫，國發會將採用大數據分析技術，強化推估效能，未來將以數位創新思維檢討人口對策，擘劃人口政策新方向。
----------------------------------------------------------------------

[48] 銀行體系資金充裕　11月活期、證券劃撥與外匯存款餘額皆創新高

▲ 11月活期性存款、外匯存款以及證券劃撥存款餘額皆創下歷史新高。（圖／路透）
記者李瑞瑾／台北報導
央行今（24）日公布11月金融情況，M1B年增率、M2年增率分別上升為7.46%及4.14%，主要受資金流入之影響。據央行數據顯示，11月活期性存款、外匯存款以及證券劃撥存款餘額，皆創下歷史新高。
央行官員表示，M2年增率續升主要是受到外資續呈淨匯入、部份企業將國外貨款匯回，致外匯存款相對成長影響。11月M2年增率也來到2016年9月以來高點（前高為2016年8月的4.51%）。
另外，1~11月M2年增率為3.4%、放款投資平均年增率為4.68%，均高於本年的經濟成長及CPI年增率預測的總和，顯示整個銀行體系資金相當充裕，足以支應經濟活動成長所需。
11月日平均貨幣總計數M1B、M2月增率分別為0.41%及0.62%，均較上月為高。累計本年1至11月M1B及M2平均年增率分別為7.12%及3.4%。
根據央行統計之全體貨幣

▲許金龍還不出保證金2億元，再遭羈押。（圖／記者吳銘峯攝）
記者吳銘峯／台北報導
前股票上櫃公司樂陞科技董事長許金龍，爆發與日商「百尺竿頭」公司違約不交割案件，二審將他判刑12年，而許金龍也籌足2億保金獲釋。但他這2億保金其實是向姊夫所借，因為無力償還，姐夫要求法院退還保金。法院審理後，24日下午裁定退還保金，但重新羈押許金龍。
樂陞科技一度榮登遊戲類股股后，2016年中傳出將被日商「百尺竿頭」公司以48億餘元公開收購，成為台灣史上遊戲業最大收購案。但最後「百尺竿頭」公司卻違約不交割，未履約付款，成為我國證券交易史上首宗公開收購違約交割案件。案發後檢調單位隨即將樂陞董事長許金龍傳喚到案，並聲請羈押獲准。
檢方最後將許金龍提起公訴，一審法院將將許金龍判刑18年、併科1億元罰金。高法院二審去年11月底宣判，仍將他重判有期徒刑12年、併科罰金1億元，並宣告沒收12.1億。高院隨即裁准許金龍2億元交保，並限制住居、出境、出海，且應於每日晚間8點至9點間至轄區派出所報到。但許金龍一直籌不出保金，直到今年3月間，才順利籌出保金，離開看守所。
但其實許金龍的2億保金乃姐夫畢華民所出具，許金龍交保後，一直無力償還保金。畢華民因此向高等法院聲請退還保金，高院24日上午開庭審理，並傳喚許金龍的哥哥許世龍到庭。由於當時保金乃許世龍與畢華民簽訂借貸契約書，由畢華民出資2億元，並擔任具保人，另外契約書上也載明，若交保後6個月內未還款，許金龍願意再被執行羈押，許金龍出監後也簽名同意擔任連帶保證人。
----------------------------------------------------------------------

[60] 智慧機械連二年產值破兆　經長：續推智慧製造產業升級

▲經濟部長沈榮津表示，未來將透過投資台灣三大方案與產創條例投資抵減，持續推動以智慧機械驅動智慧製造，以帶動產業升級。（圖／記者屠惠剛攝）
記者林淑慧／台北報導
經濟部長沈榮津今（24）日表示，在推動中小企業數位化、標竿企業智慧化政策下，智慧機械產業連續二年產值已突破兆元，未來將透過投資台灣三大方案與產創條例投資抵減，持續推動以智慧機械驅動智慧製造，以帶動產業升級。
台灣機械工業同業公會今天舉行台灣機械雲成果發表會，以發表公板機械雲，呼應去年白皮書的中期目標，經濟部長沈榮津受邀出席並發表致詞。
中

▲敏盛醫捐贈台大護理系校舍。（圖／敏盛提供）
記者姚惠茹／台北報導
盛弘醫藥（8403）旗下敏盛醫療體系今（24）日與國立台灣大學舉辦簽約儀式，由敏盛醫療體系總裁楊敏盛、執行長楊弘仁與台灣大學校長管中閔、副校長張上淳共同簽署，並以實物捐贈的方式在台大醫學院校地興建護理系校舍，預計五年內完工，將以「敏盛護理健康大樓」命名啟用。
楊敏盛及三個兒女皆是台大畢業生，因此不僅在2015年7月與台大前校長楊泮池簽約捐贈新台幣1.5億元，包括修建醫學校區的護理系館以及整修校總區的第一學生活動中心「怡仁堂」，而且已在2016年底完工啟用。
敏盛總裁楊敏盛表示，醫師及護理師是攸關全民健康福址的專業人士，需要投注更多的支持與資源，尤其近年來醫病關係不佳，醫護人員的價值被貶抑，護理人員大量流失，造成護士荒，也洐生醫院減床以致影響醫療品質等問題。
敏盛表示，新建大樓坐落於台大城中校區，基地面積約2萬3千平方公尺，基地南臨徐州路、東接紹興南路、北側為女四舍、西側為公共衛生大樓，而興建範圍內允建建築面積為424坪、允建容積樓地板面積為2,545坪，並依需求興建地上七至八層教學研究空間，地下層則為地下二至三層停車空間。
敏盛指出，其實在4年前即已捐贈1.5億元給台灣大學，除了回饋母校台灣大學、台大醫學院和台大醫院外，這次再選擇捐贈護理系校舍，更是對護理人員專業的尊重與支持，也希冀吸引更多優秀青年投入醫療護理的領域。
----------------------------------------------------------------------

[70] 東森購物公佈10大熱銷商品排行榜　 安瓶熱賣可疊106座101大樓

▲東森購物發佈年度10大熱銷商品排行榜。（圖／東森購物提供，以下同）
記者洪菱鞠／台北報導
東森購物公布2019年「10大熱銷商品排行榜」，賣出最多件數冠軍是「KOPIKO集團高機能咖啡組」，共售出68,060組、2,720,000包咖啡；從營業額來看，「東森自然美水漾超彈潤口服安瓶14天神效組」則是熱銷第1，共賣出675,000瓶、疊起來有106座101高，創造1.89億元驚人成績，前10名當中，美容保健類佔了7項，可見消費者對美及健康的重視，另3項是「英國KANGOL 80周年限定後背包」、「王品集團 Tasty西堤牛排餐」券和「王品集團 陶板屋餐券」，累積銷

▲長榮航空為優化機隊持續引進波音787夢幻客機，今年再度獲得AirlineRatings.com評選為全球最佳航空公司第八名之肯定。（圖／長榮航空提供）
記者林淑慧／台北報導
航空業搶客出新招！長榮航空今(24)日表示，隨著787夢幻客機陸續交機，機隊的運用將更具彈性，為提升軟體服務力，正式導入AI文字客服機器人，從官網或APP供民眾查詢近500項專業知識，以AI智能提供零時差的數位服務，以期提升旅客黏著度。
北美是長榮航空的主力市場之一，現有紐約、洛杉磯、芝加哥、舊金山、休士頓、西雅圖、溫哥華及多倫多共八條航線往返北美，每週可提供超過82個航班的服務，除了是台灣民眾赴美的首選，現在國外旅客搭乘的比例也逐年提升，挹注公司營運表現。
不過，北美市場的長程航線具有時差，長榮航空表示，導入文字客服機器人系統，將可因應新世代數位通路服務管道的需求，也能提高客服同仁的服務水準與效率。
▲長榮航空進入AI智能時代，運用人工智慧，提供24小時文字客服機器人的服務，提供旅客各類資訊。（圖／長榮航空提供）
長榮航空指出，文字客服機器人與公司後台系統充分整合，提供最即時與正確的資料，由AI智能客服為第一線面對旅客，提供訂位票務、班機到離時刻、特殊需求、官網及App購票諮詢、行李額度諮詢、基本機場運等近500項專業知識查詢，讓旅客可以用更直覺、彈性與自主的提問方式查詢各類服務。
如果AI智能客服無法回答問題時，旅客只要以會員身分完成登入，就可於正常上班時間轉接為專人服務，如此由AI智能客服與真人服務協同合作，將有效提升客服的效率。
展望後市，法人預估在787新世代飛機投入高運量及中長程航線下，長榮航空可望增加產品競爭力，另導入新一代票價系統，以差異化票價產品滿足不同客群，以提升品質來增加航班收益，力拚全年獲利可期。
依據財報，長榮航空今年第3季稅後純益為12.71億元，雖然仍較去年同期腰斬，但比起第2季獲利只有3,319萬元，季增率高達37.5倍；單季每股獲利0.26元，與第二季只有0.01元相比，獲利已大幅改善。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[81] 惠友新捷境　6大機能一次到位　慢工細活打造只為成就最

▲隨著中美貿易戰取得第一階段協議，台股上周一度站上12,125.9點，再創逾29年半新高。（示意圖／記者林敬旻攝）
記者紀佳妘／台北報導
中美貿易戰確定達成第一階段協議，全球股市多頭氣勢難擋，在美股持續創高下，上周台股一度站上12,125.9點，再創逾29年半新高。展望後市，法人預期，隨著外資開始進入聖誕、新年假期，台股將進入作夢行情，中長線指數仍有機會上攻。
觀察外資資金流向，根據彭博統計，上周吸金王寶座由南韓拿下，一舉獲得淨流入10.3億美元（約新台幣313億元）；其次為印度，吸金6.7億美元（約新台幣204億元）；台灣則因漲多面臨修正，外資由流入轉為流出，上周失血4.2億美元（約新台幣127億元）。
▲外資對亞洲股市買賣超金額，其中台股面臨漲多修正，上周失血4.2億美元。（圖／中信投信提供，點圖可放大）
中國信託台灣活力基金經理人周俊宏表示，就技術面觀察，指數面臨漲多修正，目前適逢聖誕節與新年假期，受到外資放假量縮影響，使得資金面稍嫌疲弱，後市可觀察外資收假後資金走向，以及觀察月線支撐是否穩固。
周俊宏指出，中美第一階段貿易協定可望於1月簽署，預期短期內沒有重要的總經議題。此外，目前財報進入空窗期，盤勢將進入作夢行情，預期2020年業績成長股以及產業題材性高的族群將扮演主流，選股上，看好5G、記憶體、iPhone等議題，預期中長線指數仍有機會上攻。
至於亞洲其他國家部分，中國信託亞太多元入息平衡基金經理人陳彥羽表示，亞太地區11月製造業PMI多數呈現增長，對於明年的經濟展望為溫和復甦，成長可期。不過，由於南韓Kospi指數已跌破年線，加上南韓財政部預估，明年經濟成長率將為2.4%，較今年成長，吸引資金流入南韓股市；至於大陸部份，目前關注電子新產品，如5G手機、基地台設備、無線充電、網路安全等族群。
----------------------------------------------------------------------

[93] 台商回台炒熱氣氛！公股銀上調放款成長目標　華銀估明年成長5%

▲華南銀行。（圖／記者吳靜君攝）
記者吳靜君／台北報導
隨著貿易戰持續、台商回台投資需求旺盛，也帶動銀行的融資市場，公股銀行紛紛調高明年的放款成長率；以一銀來說明年目標要增長7%~8%；華南金暨華南銀行董事長張雲鵬今(23)日表示，看好台商回台投資的貸

In [6]:
# 關閉瀏覽器
browser.close()